
<img align="left" style="padding-right:100px;" src="http://pvexpect.com/Driftsledelse/Dagligvarer/nmbu.png">

  **Digital Driftsledelse - IND310**

 Jesper Ørntoft Frausig, 2022


# **4.0 Areal diagram**

- [4.0 Areal diagram](#4.0)
- [4.1 Områdes diagram med fargegradient](#4.1)
- [4.2 Kumulativt tellingsdiagram](#4.2)
- [4.3 Fasetterte tetthetsestimater](#4.3)
- [4.4 Horisontgraf](#4.4)
- [4.5 Diagram med intervallvalg](#4.5)
- [4.6 Lagdelt arealdiagram](#4.6)
- [4.7 Normalisert stablet arealdiagram](#4.7)
- [4.8 Stablet tetthetsestimater](#4.8)
- [4.9 Streamgraf](#4.9)
- [4.10 Trellis Areal graf](#4.10)
- [4.11 Trellis sortert arealgraf](#4.11)

## **4.1 Områdes diagram med fargegradient**

https://altair-viz.github.io/gallery/area_chart_gradient.html

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
#Hent aktier
import yfinance as yf
def stockDownload(symbol):
  stock = yf.Ticker(symbol)
  hist = stock.history(period="5y")
  return hist

In [3]:

data=stockDownload("Mowi.OL")
data['Dato']=data.index

In [4]:
import altair as alt

alt.Chart(data).mark_area(
    line={'color':'darkgreen'},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='white', offset=0),
               alt.GradientStop(color='darkgreen', offset=1)],
        x1=1,
        x2=1,
        y1=1,
        y2=0
    )
).encode(
    alt.X('Dato:T'),
    alt.Y('Close:Q')
)

alt.Chart(...)

## **4.2 Kumulativt tellingsdiagram**

https://altair-viz.github.io/gallery/cumulative_count_chart.html

In [5]:
import altair as alt
from vega_datasets import data

source = data.movies.url

alt.Chart(source).transform_window(
    cumulative_count="count()",
    sort=[{"field": "IMDB_Rating"}],
).mark_area().encode(
    x="IMDB_Rating:Q",
    y="cumulative_count:Q"
)

alt.Chart(...)

## **4.3 Fasetterte tetthetsestimater**

https://altair-viz.github.io/gallery/density_facet.html

In [8]:
"""
import altair as alt
import pandas as pd

data = pd.io.stata.read_stata('https://github.com/ECN431/2018/blob/master/lab5/fisheries.dta?raw=true')
alt.data_transformers.disable_max_rows()
	
data['quality']=data['quality'].astype("str")
alt.Chart(data.sample(n=4000, random_state=1)).mark_area().transform_density(
    density='quality',
    groupby=['tool'],
    extent= [0, 100]
).encode(
    alt.X('age:Q'),
    alt.Y('density:Q'),
    alt.Row('tool:N')
).properties(width=300, height=200)
"""

alt.Chart(...)

## **4.4 Horisontgraf**

https://altair-viz.github.io/gallery/horizon_graph.html

## **4.5 Diagram med intervallvalg**

https://altair-viz.github.io/gallery/interval_selection.html

In [9]:

import pandas as pd
data=pd.read_csv("https://www.fiskeridir.no/Akvakultur/Tall-og-analyse/Biomassestatistikk/Biomassestatistikk-etter-fylke/_/attachment/download/422fa084-c905-43b4-9446-a050e92e80e5:1c81cbe34307c35cb79e7dc5a55aae783ab17ac7/biostat-total-flk-2019.csv", delimiter=";", error_bad_lines=False, encoding='latin-1', decimal=",")

data_selection=data[data.ARTSID=="LAKS"].groupby(['MÅNED','ÅR','MÅNED_KODE','FYLKE']).sum()
data_selection.reset_index(inplace=True)

import datetime
data_selection=data_selection.assign(**{'Date':0, 'DAY':1})

for index, måned in enumerate(data_selection.MÅNED):
  
  data_selection.MÅNED[index]=str(måned).replace('DESEMBER','DECEMBER').replace('FEBRUAR','FEBRUARY').replace('JANUAR','JANUARY').replace('JULI','JULY').replace('JUNI','JUNE').replace('MAI','MAY').replace('MARS','MARCH').replace('OKTOBER','OCTOBER')

  data_selection.Date[index] =datetime.datetime.strptime(str(data_selection.ÅR[index])+str(data_selection.MÅNED[index])+str(data_selection.DAY[index]), "%Y%B%d")

data_selection.MÅNED=data_selection.MÅNED.astype(str)
data_selection['timestamp'] = pd.to_datetime(data_selection.Date)




/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
import altair as alt

brush = alt.selection(type='interval', encodings=['x'])

base = alt.Chart(data_selection.drop(['Date','MÅNED'], axis=1)).mark_area().encode(
    x="timestamp:T",
    y="BIOMASSE_KG:Q",
    color="FYLKE:N"
).properties(
    width=600,
    height=200
)

upper = base.encode(
    alt.X('timestamp:T', scale=alt.Scale(domain=brush))
)

lower = base.properties(
    height=60
).add_selection(brush)

(upper & lower)#.save("Areal diagram.html")


alt.VConcatChart(...)

## **4.6 Lagdelt arealdiagram**

https://altair-viz.github.io/gallery/layered_area_chart.html

In [12]:
import altair as alt

data=pd.read_csv("https://www.fiskeridir.no/Akvakultur/Tall-og-analyse/Biomassestatistikk/Biomassestatistikk-etter-fylke/_/attachment/download/422fa084-c905-43b4-9446-a050e92e80e5:1c81cbe34307c35cb79e7dc5a55aae783ab17ac7/biostat-total-flk-2019.csv", delimiter=";", error_bad_lines=False, encoding='latin-1', decimal=",")
data_selection=data[data.ARTSID=="LAKS"].groupby(['MÅNED','ÅR','MÅNED_KODE','FYLKE']).sum()
data_selection.reset_index(inplace=True)

import datetime
data_selection=data_selection.assign(**{'Date':0, 'DAY':1})

for index, måned in enumerate(data_selection.MÅNED):
  
  data_selection.MÅNED[index]=str(måned).replace('DESEMBER','DECEMBER').replace('FEBRUAR','FEBRUARY').replace('JANUAR','JANUARY').replace('JULI','JULY').replace('JUNI','JUNE').replace('MAI','MAY').replace('MARS','MARCH').replace('OKTOBER','OCTOBER')

  data_selection.Date[index] =datetime.datetime.strptime(str(data_selection.ÅR[index])+str(data_selection.MÅNED[index])+str(data_selection.DAY[index]), "%Y%B%d")

data_selection.MÅNED=data_selection.MÅNED.astype(str)
data_selection['timestamp'] = pd.to_datetime(data_selection.Date)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
alt.Chart(data_selection.drop(['Date','MÅNED'], axis=1)).mark_area(opacity=0.3).encode(
    x="timestamp:T",
    y=alt.Y("BIOMASSE_KG:Q", stack=None),
        color="FYLKE:N"

)

alt.Chart(...)

## **4.7 Normalisert stablet arealdiagram**

https://altair-viz.github.io/gallery/normalized_stacked_area_chart.html

In [14]:
import altair as alt

data=pd.read_csv("https://www.fiskeridir.no/Akvakultur/Tall-og-analyse/Biomassestatistikk/Biomassestatistikk-etter-fylke/_/attachment/download/422fa084-c905-43b4-9446-a050e92e80e5:1c81cbe34307c35cb79e7dc5a55aae783ab17ac7/biostat-total-flk-2019.csv", delimiter=";", error_bad_lines=False, encoding='latin-1', decimal=",")
data_selection=data[data.ARTSID=="LAKS"].groupby(['MÅNED','ÅR','MÅNED_KODE','FYLKE']).sum()
data_selection.reset_index(inplace=True)

import datetime
data_selection=data_selection.assign(**{'Date':0, 'DAY':1})

for index, måned in enumerate(data_selection.MÅNED):
  
  data_selection.MÅNED[index]=str(måned).replace('DESEMBER','DECEMBER').replace('FEBRUAR','FEBRUARY').replace('JANUAR','JANUARY').replace('JULI','JULY').replace('JUNI','JUNE').replace('MAI','MAY').replace('MARS','MARCH').replace('OKTOBER','OCTOBER')

  data_selection.Date[index] =datetime.datetime.strptime(str(data_selection.ÅR[index])+str(data_selection.MÅNED[index])+str(data_selection.DAY[index]), "%Y%B%d")

data_selection.MÅNED=data_selection.MÅNED.astype(str)
data_selection['timestamp'] = pd.to_datetime(data_selection.Date)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
import altair as alt

alt.Chart(data_selection.drop(['Date','MÅNED'], axis=1)).mark_area(opacity=0.3).encode(
    x="timestamp:T",
    y=alt.Y("BIOMASSE_KG:Q", stack='normalize'),
        color="FYLKE:N"

)

alt.Chart(...)

## **4.8 Stablet tetthetsestimater**

https://altair-viz.github.io/gallery/density_stack.html

## **4.9 Streamgraf**

## **4.10 Trellis Areal graf**

## 4.11 **Trellis sortert arealgraf**